# Introduction
---
기계 학습 방법이 제대로 작동하려면 어떤 것들이 필요할까요? 원론적인 답을 해보자면 잘 설계된 모델, 학습을 감당할 수 있는 하드웨어, 풀고자 하는 문제에 대한 다양한 데이터가 필요합니다. 어느 하나 덜 중요한 것이 없지만, 기계 학습 방법론이 오늘날처럼 대두되기 이전에는 주로 새로운 모델을 만들어 내는 것에 더 중점을 두었습니다. 인간이 세상을 지각하는 방법을 모사할 수 있다면 영상인식이나, 음성인식 같은 문제들이 자연스럽게 풀릴 수 있을 것이라 생각한 것입니다. 여기서 놓치고 있던 것은 무엇일까요? 바로 데이터입니다. 우리에게 있어서 임의의 자극(시각, 음성 등)을 알아채는 과정은 너무나 자연스럽게 느껴지기 때문입니다. 하지만 유아기를 돌아보면, 새로운 자극을 만날 때마다 호기심을 갖고 부모나 또 다른 누군가에게 그것이 무엇인지 끊임없이 물어봤을 것입니다. 이렇게 아이가 세상을 '보는 법'을 배워나가는 과정에서 착안해 [ImageNet](http://www.image-net.org/)이라는 방대한 데이터베이스를 구축합니다.

>참고 자료
* Fei-Fei Li의 [TED 강연](https://www.ted.com/talks/fei_fei_li_how_we_re_teaching_computers_to_understand_pictures)

## ILSVRC
---
[ILSVRC(ImageNet Large Scale Visual Recognition Competition)](http://image-net.org/challenges/LSVRC/)는 ImageNet의 일부(1000개의 부류, 각 부류당 약 1000개의 이미지)를 사용해 맞는 부류를 찾아내는 대회입니다. 1순위 오류율(top-1 error rate)와 5순위 오류율(top-5)로 성능을 측정하는데 2010년, 2011년 우승 팀의 5순위 오류율은 각각 28.2%, 25.8%였습니다.

In [1]:
import torch
import torch.nn as nn
import numpy as np

# AlexNet
---
AlexNet은 [ImageNet Classification with Deep Conolutional Neural Networks](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks)의 저자 중 한 명인 Alex Krizhevsky의 이름을 따왔습니다. 5개의 컨볼루션층(convolution layer)와 분류를 위한 3개의 완전연결층(fully connected layer)로 이루어져 있습니다. 입력으로는 ImageNet의 데이터(3\*244\*244의 RGB 채널 이미지)를 사용했고, 첫 번째 컨볼루션층에서 3\*11\*11 필터 96개를 사용했습니다.

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, n_classes=2):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), 
                                    nn.BatchNorm2d(96), 
                                    nn.ReLU(), 
                                    nn.MaxPool2d(3, stride=2)
                                  )
        self.conv2 = nn.Sequential(nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=1), 
                                    nn.BatchNorm2d(256), 
                                    nn.ReLU(), 
                                    nn.MaxPool2d(3, stride=2)
                                  )
        self.conv3 = nn.Sequential(nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1), 
                                    nn.ReLU()
                                  )
        self.conv4 = nn.Sequential(nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU()
                                  )
        self.conv5 = nn.Sequential(nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU()
                                  )
        self.fc1 = nn.Sequential(nn.Linear(4096, 4096), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(4096, 4096), nn.ReLU())
        self.fc3 = nn.Sequential(nn.Linear(4096, n_classes), nn.Softmax())
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

##  VGGNet
---
$VGGNet$을 한 마디로 요약하면 '더 작은 필터, 더 깊은 층'이 되겠습니다. 3\*3 필터를 사용해 더 깊은 신경망을 구성했고, 13개의 컨볼루션층과 3개의 완전연결층을 사용했습니다($VGG-16$의 경우).

In [ ]:
class VGG16(nn.Module):
    def __init__(self, n_classes=2):
        super(VGG16, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.MaxPool2d(2, stride=2)
                                  )
        self.conv2 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.MaxPool2d(2, stride=2)
                                  )
        self.conv3 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1), 
                                   nn.MaxPool2d(2, stride=2)
                                  )
        self.conv4 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.MaxPool2d(2, stride=2)
                                  )
        self.conv5 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), 
                                   nn.ReLU(), 
                                   nn.MaxPool2d(2, stride=2)
                                  )
        self.fc = nn.Sequential(nn.Linear(4096, 4096), 
                                nn.ReLU(), 
                                nn.Linear(4096, 4096), 
                                nn.ReLU(), 
                                nn.Linear(4096, n_classes), 
                                nn.Softmax())
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        
        return out

## GoogLeNet
---
$Network In Network$($NIN$)에서 영감을 얻은 $Inception module$을 사용했습니다. 기존 컨볼루션 연산이 필터와 입력값의 선형조합으로 이루어진 반면 $NIN$에서는 MLP가 들어갑니다.

In [ ]:
class GoogLeNet(nn.Module):
    def __init__(self, n_classes=2):
        super(GoogLeNet, self).__init__()